<a href="https://colab.research.google.com/github/harnalashok/deeplearning/blob/main/intel_images_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last amended: 16th October, 2021
# Ref: https://www.kaggle.com/puneet6060/intel-image-classification
#     https://www.kaggle.com/accountstatus/using-keras-cnn-on-intel-data

In [ ]:
# 0.0 We need to install h5py to save our model:

! pip install pyyaml h5py

## Call libraries

In [ ]:
# 1.0 Call libraries
import pandas as pd
import numpy as np

# 1.1
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import save_model
from tensorflow.keras.layers import Dense,Convolution2D,MaxPooling2D,Flatten,Conv2D,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# 1.2
from PIL import Image
import matplotlib.pyplot as plt
import os


## Design model

In [ ]:
# Creating a simple sequential model
# 2.0
model=Sequential()
model.add(Convolution2D(
                        32,
                        (3,3),
                        input_shape=(64,64,3),
                        activation='relu'
                        )
           )
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(
                        64,
                        (3,3),
                        activation='relu'
                        )
           )

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(units=6,activation='softmax'))

In [ ]:
# 2.1 Get model summary
model.summary()

In [ ]:
# 2.2 Plot model:

tf.keras.utils.plot_model(
                           model,
                           to_file="model.png",
                           show_shapes=False,
                           show_layer_names=True,
                           rankdir="TB",
                           expand_nested=False
                          )

In [ ]:
# 2.3 Compile model:

model.compile(
              optimizer= 'adam',
              loss= 'categorical_crossentropy',
              metrics= ['accuracy']
              )

## Image augmentation

In [ ]:
# 3.0 Instantiate ImageDataGenerator:

train_datagen = ImageDataGenerator(
                                   rescale = 1./255, 
                                   #shear_range = 0.2,    # Increase delay
                                   #zoom_range = 0.2,     # increase delay
                                   horizontal_flip = True
                                   )

In [ ]:
# 3.1 Mount gdrive
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
# 3.2 Where is our train and test data:

train_path = '/gdrive/MyDrive/Colab_data_files/intel_images/seg_train' 
test_path = '/gdrive/MyDrive/Colab_data_files/intel_images/seg_test'

In [ ]:
# 3.3 Create training data generator:

training_set = train_datagen.flow_from_directory(
                                                  train_path,
                                                  target_size = (64, 64), 
                                                  batch_size = 32
                                                 )


In [ ]:
# 3.4 This is our test data generator:

test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(
                                            test_path,
                                            target_size = (64, 64),
                                            batch_size = 32
                                            )

In [ ]:
# 3.5 How many classes are there:
len(set(training_set.classes))

# 3.6 And which ones:

l=os.listdir(train_path)
l.sort()

## Train model

In [ ]:
# 4.0 Fit the model:
fitted_model = model.fit_generator(
                                    training_set,
                                    steps_per_epoch = 100,      # select 50, if short of time
                                    epochs = 10,                # Select 5, if short of time
                                    validation_data = test_set,
                                    validation_steps = 10
                                   )

## Save model

In [ ]:
# 5.0 Folder to which trained model will be saved
! mkdir /gdrive/MyDrive/Colab_data_files/intelmodel

# 5.1 This will be our model file:
pathToSave = "/gdrive/MyDrive/Colab_data_files/intelmodel/intel_images_model.h5"

In [ ]:
# 5.2 Save now in h5format

model.save(pathToSave)

## Load model & experiment

In [ ]:
# 6.0 Load saved model:

del model
mymodel = keras.models.load_model(pathToSave)

In [ ]:
# 6.1 Here are few images to be tested:

image_path_forest = "/gdrive/MyDrive/Colab_data_files/intel_images/seg_train/forest/10007.jpg"
image_path_building = "/gdrive/MyDrive/Colab_data_files/intel_images/seg_train/buildings/10006.jpg"
image_path_glacier = "/gdrive/MyDrive/Colab_data_files/intel_images/seg_train/glacier/10.jpg"
image_path_mountain = "/gdrive/MyDrive/Colab_data_files/intel_images/seg_train/mountain/10008.jpg"
image_path_sea = "/gdrive/MyDrive/Colab_data_files/intel_images/seg_train/sea/10016.jpg"
image_path_street = "/gdrive/MyDrive/Colab_data_files/intel_images/seg_train/street/10015.jpg"

In [ ]:
# 6.2 Define a prediction function:

def predict(image_path): 
    model = keras.models.load_model(pathToSave)
    image = load_img(image_path, target_size=(64, 64))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    #image = preprocess_input(image)
    yhat = model.predict(image)
    classes_x=np.argmax(yhat,axis=1)
    l=os.listdir(train_path)
    l.sort()
    return l[classes_x[0]]

In [ ]:
# 6.3 Make predictions:

predict(image_path_street)

In [ ]:
####### I am done ##############